In [1]:
# Dependencies and Setup
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [2]:
# Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

In [3]:
# Visit the NASA Mars News Site
browser.visit('https://mars.nasa.gov/news')

In [4]:
# Parse Results HTML with BeautifulSoup
# Find Everything Inside:
#   <ul class="item_list">
#     <li class="slide">
html = browser.html
news_soup = BeautifulSoup(html, "html.parser")
slide_element = news_soup.select_one("ul.item_list li.slide")

In [5]:
slide_element.find("div", class_="content_title")

<div class="content_title"><a href="/news/8522/nasas-curiosity-rover-finds-an-ancient-oasis-on-mars/" target="_self">NASA's Curiosity Rover Finds an Ancient Oasis on Mars</a></div>

### NASA Mars News

In [6]:
# Scrape the Latest News Title
# Use Parent Element to Find First <a> Tag and Save it as news_title
news_title = slide_element.find("div", class_="content_title").get_text()
print(news_title)

NASA's Curiosity Rover Finds an Ancient Oasis on Mars


In [7]:
# Scrape the Latest Paragraph Text
news_paragraph = slide_element.find("div", class_="article_teaser_body").get_text()
print(news_paragraph)

New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate.


### JPL Mars Space Images - Featured Image

In [8]:
# Visit the NASA JPL (Jet Propulsion Laboratory) Site
executable_path = {"executable_path": "./chromedriver"}
browser = Browser("chrome", **executable_path)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [9]:
# Ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [10]:
# Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

In [11]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")

In [12]:
img_url = image_soup.select_one("figure.lede a img").get("src")
img_url

'/spaceimages/images/largesize/PIA01322_hires.jpg'

In [13]:
# Use Base URL to Create Absolute URL
img_url = f"https://www.jpl.nasa.gov{img_url}"
print(img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01322_hires.jpg


### Mars Weather

In [14]:
# Visit the Mars Weather Twitter Account
executable_path = {"executable_path": "./chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [15]:
# Parse Results HTML with BeautifulSoup
html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")

In [16]:
# Find a Tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_soup.find("div", 
                                       attrs={
                                           "class": "tweet", 
                                            "data-name": "Mars Weather"
                                        })

In [17]:
# Search Within Tweet for <p> Tag Containing Tweet Text
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
print(mars_weather)

InSight sol 313 (2019-10-14) low -101.4ºC (-150.6ºF) high -27.2ºC (-16.9ºF)
winds from the SSE at 5.5 m/s (12.4 mph) gusting to 22.5 m/s (50.3 mph)
pressure at 7.20 hPapic.twitter.com/2moNAouxXa


### Mars Facts

In [18]:
# Visit the Mars Facts Site Using Pandas to Read
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
print(mars_df)
mars_df.columns=["Description", "Mars_Value", "Earth_Value"]
mars_df.set_index("Description", inplace=True)
mars_df

  Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:    -153 to 20 °C      -88 to 58°C


,Mars_Value,Earth_Value
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-153 to 20 °C,-88 to 58°C


### Mars Hemisphere

In [19]:
# Visit the USGS Astrogeology Science Center Site
executable_path = {"executable_path": "./chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [20]:
hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find Element on Each Loop to Avoid a Stale Element Exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find Sample Image Anchor Tag & Extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere Object to List
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate Backwards
    browser.back()

In [21]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]